# Updating the raw table to have all the historical data

Installing Requirements

In [3]:
%%sh 
pip install pymc3

  Created wheel for theano: filename=Theano-1.0.4-cp37-none-any.whl size=2667178 sha256=9f4cf2ea088477af37e7f8844b44d13abc546cb9ceec8b9cd63a1680699b988f
  Stored in directory: /home/jovyan/.cache/pip/wheels/88/fb/be/483910ff7e9f703f30a10605ad7605f3316493875c86637014
Successfully built theano
  Found existing installation: setuptools 41.0.1
    Uninstalling setuptools-41.0.1:
      Successfully uninstalled setuptools-41.0.1


Importing Requirements

In [37]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns;sns.set()
from IPython.core.pylabtools import figsize

import requests 
pd.options.display.max_rows = 999
import io
from datetime import datetime

# PyMC3 for Bayesian Inference
import pymc3 as pm

import scipy
from scipy import stats
from scipy.stats import percentileofscore

# Standard ML Models for comparison
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

np.random.seed(42)
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['figure.figsize'] = (9, 9)

# Importing Utilities 
from utils import *
from importlib import reload

hide_toggle()


Starting Spark Session

In [6]:
# setup-- 
import pandas as pd
import numpy as np

import pyspark
from pyspark import SQLContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer

spark = SparkSession.builder.getOrCreate()
sqlCtx = spark._wrapped

hide_toggle()

Starting Splice Context

In [8]:
from splicemachine.spark.context import PySpliceContext
from splicemachine.ml.management import MLManager

splice=PySpliceContext(spark)
manager = MLManager(splice)

hide_toggle()

Tracking Model Metadata on MLFlow Server @ http://splicedb-mlflow-0.splicedb-mlflow.mldemo.svc.cluster.local:5001


In [38]:
us_daily = pd.DataFrame.from_dict(requests.get('https://covidtracking.com/api/us/daily').json())
states = pd.DataFrame.from_dict(requests.get('https://covidtracking.com/api/states').json())
states_daily = pd.DataFrame.from_dict(requests.get('https://covidtracking.com/api/states/daily').json())

In [39]:
states_daily_spark = sqlCtx.createDataFrame(states_daily)

In [40]:
def set_df_columns_nullable(df, column_list, nullable=True):
    for struct_field in df.schema:
        if struct_field.name in column_list:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

def replace(column):
    return F.when(column != np.nan, column).otherwise(F.lit(None))

states_daily_spark = set_df_columns_nullable(states_daily_spark,states_daily_spark.columns)

for _col,_type in states_daily_spark.dtypes:
    if _col in ['state','dateChecked']:
        continue
    else:
        states_daily_spark = states_daily_spark.withColumn(_col, replace(F.col(_col)))
        

states_daily_spark.printSchema()
states_daily_spark.show()

root
 |-- date: long (nullable = true)
 |-- state: string (nullable = true)
 |-- positive: double (nullable = true)
 |-- negative: double (nullable = true)
 |-- pending: double (nullable = true)
 |-- hospitalized: double (nullable = true)
 |-- death: double (nullable = true)
 |-- total: long (nullable = true)
 |-- dateChecked: string (nullable = true)
 |-- totalTestResults: long (nullable = true)
 |-- deathIncrease: double (nullable = true)
 |-- hospitalizedIncrease: double (nullable = true)
 |-- negativeIncrease: double (nullable = true)
 |-- positiveIncrease: double (nullable = true)
 |-- totalTestResultsIncrease: double (nullable = true)

+--------+-----+--------+--------+-------+------------+-----+-----+--------------------+----------------+-------------+--------------------+----------------+----------------+------------------------+
|    date|state|positive|negative|pending|hospitalized|death|total|         dateChecked|totalTestResults|deathIncrease|hospitalizedIncrease|negativeIn

In [41]:
splice.createTable(states_daily_spark,'COVID.TEMP',drop_table = True)
splice.insert(states_daily_spark,'COVID.TEMP')

Droping table COVID.TEMP
CREATE TABLE COVID.TEMP(
DATE BIGINT,
STATE VARCHAR(5000),
POSITIVE DOUBLE,
NEGATIVE DOUBLE,
PENDING DOUBLE,
HOSPITALIZED DOUBLE,
DEATH DOUBLE,
TOTAL BIGINT,
DATECHECKED VARCHAR(5000),
TOTALTESTRESULTS BIGINT,
DEATHINCREASE DOUBLE,
HOSPITALIZEDINCREASE DOUBLE,
NEGATIVEINCREASE DOUBLE,
POSITIVEINCREASE DOUBLE,
TOTALTESTRESULTSINCREASE DOUBLE)


In [42]:
%%sql 
DROP TABLE IF EXISTS COVID.RAW_DATA;

CREATE TABLE COVID.RAW_DATA AS SELECT * FROM COVID.TEMP;

SELECT COUNT(*) FROM COVID.RAW_DATA;

In [43]:
%%sql 
DROP TABLE COVID.TEMP;

Query executed successfully. Affected rows : 0

In [44]:
%%sql 
SELECT * FROM COVID.RAW_DATA